# Functions in Julia
---
* Created on 30 Aug 2023
* Created by Yooshin Oh (stevenoh0908@snu.ac.kr)
---
* <span class="mark">Documentation: https://docs.julialang.org/en/v1/manual/functions/</span>

## Introduction

* Julia에서 Function은 `Tuple` -> `Return Value`로 Mapping해주는 객체로 정의된다.
* Julia에서의 함수는 수학적 함수(Mathematical Functions)와는 다른데, 왜냐하면 수학적 함수와는 달리 프로그램 실행 중에 '변경될 수 있고', 따라서 Program의 Global State에 영향을 미칠 수 있기 때문이다.

## Function Definition

* Julia에서는 다음과 같은 문법으로 함수를 정의한다. (Bash-like style)
    ```julia
    function f(x, y)
        x + y
    end
    ```
* 즉, 다음과 같은 Template으로 함수를 정의한다.
    ```julia
    function function_name(arg_seperated_by_comma)
        expressions
    end
    ````

In [1]:
function f(x,y)
    x + y
end

f (generic function with 1 method)

* 이상의 함수는 `x`, `y`의 2개의 Argument를 받아서, `x+y`를 return한다.

* 또는 다음과 같이 대입 연산자를 이용하여 곧바로 함수의 Return 값을 명시하는 식인 **Assignment Form**으로 함수를 정의할 수도 있다.

In [2]:
f(x,y) = x + y

f (generic function with 1 method)

* 간단하거나 짧은 함수는 Julia에서 위와 같이 Assignment Form으로 정의하는 것이 훨씬 낫다. (수학적으로도 좀 괜찮기도 하고)

* 함수 자체가 하나의 객체이므로, 함수도 대입 연산자를 사용하여 복사될 수 있다.

In [4]:
f(x,y) = x + y
g = f
g(2,3)

5

* Julia는 Identifier로 Unicode 표준을 따르는 문자 체계를 사용하므로, Unicode도 함수 이름으로 사용될 수 있다.

In [6]:
∑(x,y) = x + y
∑(2,3)

5

## Function Call

* Julia에서 함수의 호출 방법은 Python과 동일하다.

In [3]:
f(x,y) = x + y
f(2,3)

5

## Function Argument Passing Behaviour

* Julia Function에 대해 함수 인자는 Call-by-value가 아닌 Call-by-reference와 같은 형태로 전달된다. (Pass-by-sharing이라 하기도 한다)
* 즉, Julia Function에 인자를 전달하는 경우, 이들 인자에 상응하는 변수들에 Function의 매개변수라는 새로운 Identifier가 임시로 부여되는 것이다. (*Function arguments themelves act as new variable bindings -> new "names" that can refer to values*)
* 따라서 Julia Function 내부에서 변수 값을 수정하면, 그 변경 사항이 외부에 반영된다. (Call-by-reference)
* 이러한 동작은 LISP, Python, Ruby, Perl 등등과 정확히 같은 방식이다.

In [7]:
function f(x, y)
    x[1] = 42 # mutates x
    y = 7 + y # new binding for y, no mutation
    return y
end

f (generic function with 1 method)

* 위 경우 함수에 전달된 `x`는 Mutate되지만, 2번째 line에 의하여 함수에 전달된 `y`는 Mutate되지 않고, Integer가 Immutable Type이므로 새로운 7 + y 값이 assign되어 `y`라는 이름을 새로 부여받게 된다. 따라서 원래 함수에 인자로 전달한 `y` 변수의 값은 변경되지 않는다.

In [8]:
a = [4, 5, 6]
b = 3
f(a, b) # returns 7 + b => 10

10

In [9]:
a # a changed, due to function f, mutated a.

3-element Array{Int64,1}:
 42
  5
  6

In [10]:
b # not changed.

3

* <span class="birk">**Naming Convection**</span> : 보통 함수 안에서 Argument의 Mutation이 일어나는 경우, 함수의 Identifier의 끝을 `!`로 끝내는 것이 Julia에서 권장된다.

## Argument-type Declarations for Functions

* **Julia에서는 Function Argument 뒤에 `::TypeName`을 붙여 그 변수의 Type을 지정할 수 있다.** 이 문법은 임의의 Variable에 대해서도 적용된다.

In [ ]:
# Fibonacci Number (Recursively)
fib(n::Integer) = n <= 2 ? one(n) : fib(n-1) + fib(n-2)
# ::Integer로 n의 Type이 지정되어 있으므로, n은 Integer 또는 그 Sub-type으로만 한정된다.

* <span class="mark">**Note**</span> **Julia에서 Function에 대한 Argument-type Declaration은 Performance에 보통 영향이 없다.** Julia는 Argument에 대한 Type이 선언되어 있든지 아니든지 간에, 실제로 Function Call에서 Argument로 넘어간 값의 Type에 따라서 Function을 그때그때 Compile하고 Optimmize하기 때문이다.
* 따라서 보통 Julia에서는 Performance Issue보다는 다음과 같은 이유로 Argument Type을 명시해주게 된다.

> **Julia에서 Argument-type Declaration을 하는 이유**
>
> * **Dispatch**: Argument의 Type에 따라 다른 동작 · 연산을 수행해야 하는 경우가 있기 때문에, 이 경우는 Function Argument의 Type을 특정으로 한정해야 한다. 이 경우를 위해 Argument-type Declaration을 쓴다.
> * **Correctness**: Argument Type을 한정하여 항상 올바른 대답을 낼 것을 보장하기 위해 Function Argument-type Declaration을 쓸 수도 있다. 이를테면, 상기 피보나치 함수 예제에서 `fib(n) = n <= 2 ? one(n) : fib(n-1) + fib(n-2)`로 정의하면서 `n`의 Argument-type을 한정하지 않은 경우, `fib(1.5)`를 실행했을 때 Nonsensical한 `1.0`을 돌려줄 수도 있는 문제가 있다.
> * **Clarity**: Argument Type을 명시해주면 Function에서 Expect Argument Type을 강제하거나 명시해줄 수 있으므로 도움이 된다.

* 그러나 함수에서 Argument Type을 지나치게 강하게 제한하지 않도록 주의해야 한다. Function의 Argument Type을 지나치게 제한하면, 유사한 연산을 하는 다른 Type의 연산에 대해서 함수가 재사용될 수 없게 된다.
    * 이를테면, 위 피보나치 함수의 예제 같은 경우 `n::Int`와 같이 지나치게 강하게 Argument Type을 제한하는 경우, 피보나치 항이 너무 커져 `Int64`의 범위를 넘어 `BigInt`가 되는 경우 Exception이 던져지는 문제가 난다. `n::Integer`와 같이 적당한 수준으로 Argument Type을 제한해야, `Integer`의 sub-type인 `Int8`, `Int16`, `Int32`, `Int64`, `BigInt` 모두를 Accept하므로 임의의 피보나치 항을 계산할 수 있다.
* 앞서 강조했듯, **Julia에서 Type Declaration**은 성능에 영향을 주지 않는다. 따라서 만약 Argument의 Type에 대해 확신이 없다면, Type Declaration을 하지 않는 것이 차라리 낫다. 나중에 Type에 대한 정보가 확실해질 때, Type Specification을 나중에 해도 된다. (Julia가 알아서 Type을 Runtime 중에 보고 Optimization해서 Function-Compile을 한다)

## The `return` Keyword

* **기본적으로 Julia Function에서는 Last Expression의 값이 Evaluate되고 Return되지만,** 다른 프로그래밍 언어에서 그러한 것처럼 `return` 키워드를 사용해 return 값을 그 즉시 돌려주는 것도 가능하다.

In [ ]:
function g_func(x,y)
    println(x+y)
    x + y # The last expression of the function, will be returned by the default in Julia.
end

In [12]:
function g_func(x,y)
    return x * y # The return keyword return its specified value immediately.
    x + y # since in the previous line the function will be terminated at once, therefore this statement will never be evaluated or returned.
end

g_func (generic function with 1 method)

In [13]:
g_func(2,3) # 2 * 3 will be returned.

6

* 물론 `return` 키워드 없이 항상 Julia는 Function에서의 Last Expression을 항상 return하므로 굳이 `return` 키워드는 간단한 함수에는 필요하지 않다. 그러나 실행 도중, 즉 함수의 가장 끝이 아니라 특정 조건에 해당할 때 즉시 반환이 일어나야 하는 경우 `return` 키워드가 여타 프로그래밍 언어에서 그러한 것처럼 유용할 수 있다.

* 다음의 직각을 낀 두 변의 길이가 `x`, `y`일때, Overflow 없이 직각 삼각형의 빗변 길이를 계산하는 함수 `hypot(x,y)` 정의가 대표적인 예시이다.

In [14]:
function hypot(x,y)
    x = abs(x)
    y = abs(y)
    if x > y
        r = y / x
        return x * sqrt(1 + r * r)
    end
    if y == 0
        return zero(x)
    end
    r = x / y
    return y * sqrt(1 + r * r) # 여기의 return 키워드는 생략 가능하지만, Clarity를 위해 써주는 것이 좋다.
end

hypot (generic function with 1 method)

In [15]:
hypot(3, 4)

5.0

## Return Type Specification

* Julia에서는 Function Signature 뒤에 `::TypeName`, 즉 Julia의 전형적인 Type Specification 문법을 따라 Return Type을 명시해줄 수 있다.
    * 이 경우 항상 Return되는 값을 지정된 Type으로 변환하여 반환하게 된다.

In [17]:
function g_func(x,y)::Int8
    return x * y
end
typeof(g_func(1,2))

Int8

* 다만 Return Type Declaration은 Julia에서 잘 쓰지 않는다. 보통은 Return Type Declaration으로 반환값의 Type을 강제하기보다는, Function 자체를 type-stable하게 만들어서 Julia Compiler가 자동으로 Return Type을 일정하게 하게끔 만드는 것이 일반적이다.

## Returning `nothing`
* C에서 `void` Return-Type을 가지는 Function처럼, 반환값이 **없는** 함수의 경우, Julia에서는 `Core.Nothing`을 반환하는 것이 Convection이다.

In [18]:
function printx(x)
    println("x = $x")
    return nothing # 그러나 println이 반환값이 이미 없으므로(nothing 반환) 굳이 이 line이 필요하지는 않다. l(Julia Function은 last expression이 evaluate되어 자동 반환됨에 주의)
end

printx (generic function with 1 method)

* C언어에서처럼, `return` 키워드 자체만 사용하는 경우 자동으로 `nothing`을 return한다. 즉, 아무 return값이 없는 경우 `return`문만 사용해도 된다.
* 또한, Julia에서는 항상 Last Expression이 Evaluate되어 Return되므로, `nothing`만을 last expression으로 써도 같은 효과를 낼 수 있다.

## Operators in Julia (in manner of Functions)

* Julia에서는 대부분의 Operator들은 Special Syntax를 가지는 Function으로 취급할 수 있다.
    * 단, Short-Circuit Evaluation이 동반되는 `&&`나 `││` 연산자의 경우는 Operand가 먼저 모두 Evaluate되어서는 안 되므로 예외적으로 안 된다.
* 즉, Julia에서는 Function Call을 일반적인 Function Call의 Form으로도 할 수 있고 (`function_name(args)`, Infix Form) 연산자처럼 사용할 수도 있다. (제한이 좀 있긴 하지만)

In [19]:
1 + 2 + 3

6

In [20]:
+(1, 2, 3) # since addition function, + is just a function, you can call it with infix notation

6

In [24]:
add_func = +
add_func(1, 2, 3)

6

* 즉, Julia Compiler는 `1 + 2 + 3`을 실행하면 자동으로 이를 `+(1, 2, 3)`으로 Parsing하여 Function Call을 한다.

## Operators with Special Corresponding Function Names

* Julia에서 몇몇 함수나 Expression은 Parsing되었을 때 호출되는 상응 함수(Corresponding Function)의 이름이 비직관적(non-obvious)인 경우가 있다.
* 그 예시는 다음과 같다.

### hcat (`Base.hcat`): `[ ... ... ]`

* **Usage**: `hcat(arg1, arg2, ...)`
* **Desc**: 주어진 배열들이나 숫자들을 수평 방향으로 이어붙인다. `cat(A, ...; dims=2)`와 같고, `[a b c]`와 같다. (Stands for *Horizontal Concatenation*)

In [27]:
hcat([1, 2], [3, 4], [5, 6])
# 상기 call은 3개의 열벡터를 수평 방향으로 합침.
# [1, 2]는 2x1 열벡터이므로 다음의 결과를 돌려줌
# 1 3 5
# 2 4 6

2×3 Array{Int64,2}:
 1  3  5
 2  4  6

In [28]:
[[1, 2] [3, 4] [5, 6]] # 위와 같다. Syntax상.
# This will be automatically parsed into hcat([1, 2], [3, 4], [5, 6])

2×3 Array{Int64,2}:
 1  3  5
 2  4  6

In [30]:
hcat([1, 2], [3 4]) # If shape does not match, ArgumentError will be raised.

ArgumentError: [91mArgumentError: number of rows of each array must match (got (2, 1))[39m

### vcat (`Base.vcat`): `[ ... ; ... ]`

* **Usage**: `vcat(arg1, arg2, ...)`
* **Desc**: 주어진 배열들이나 숫자들을 수직 방향으로 이어붙인다. `cat(A, ...; dims=1)`와 같고, (Julia 1.4.1.에서는. 상위 버전에서는 이 다음 표현은 수평 방향으로 이어붙임을 지시하는 것으로 변경됨) `[a;; b;; c]`, 또는 `[a; b; c]`와 같다. (Stands for *Vertical Concatenation*)

In [35]:
vcat([1, 2], [3 ,4])
# 상기 call은 두 2x1 열벡터를 수직 방향으로 이어붙이므로, 결과물이 4x1 열벡터이고
# 1
# 2
# 3
# 4 를 돌려준다.

4-element Array{Int64,1}:
 1
 2
 3
 4

In [38]:
[[1, 2]; [3, 4]] # 이상은 정확히 vcat([1, 2], [3, 4])를 호출하는 것으로 자동 parsing된다.

4-element Array{Int64,1}:
 1
 2
 3
 4

In [39]:
vcat([1, 2], [3 4]) # if shapes do not match, an ArgumentError will be raised.

ArgumentError: [91mArgumentError: number of columns of each array must match (got (1, 2))[39m

### hvcat (`Base.hvcat`): `[ ... ... ; ... ... ]`

* **Usage**: `hvcat(blocks_per_row_as_tuple, values...)`
* **Desc**: 주어진 배열들이나 숫자들을 제시한 `blocks_per_row_as_tuple`의 모양에 따라 수평 · 수직으로 이어붙인다. 반드시 `blocks_per_row_as_tuple`은 각 행에 들어갈 원소의 개수를 순차대로 나열한 Tuple이어야 하고, 넘겨주는 값의 개수와 그 총 합이 같아야 한다. `[a b c; d e f]`는 자동으로 `hvcat((3, 3), a, b, c, d, e, f)`로 Parsing된다.

In [41]:
a1, a2, a3, a4, a5, a6 = 1, 2, 3, 4, 5, 6
[a1 a2 a3; a4 a5 a6] # Julia에서는 Matrix를 생성할 때 공백으로 열 구분, ;로 행 구분을 한다.
# Comma로 구분하는 경우는 Array가 되고, 열 벡터로 취급된다.

2×3 Array{Int64,2}:
 1  2  3
 4  5  6

* 이상의 코드는 다음과 정확히 같다.

In [42]:
hvcat((3, 3), a1, a2, a3, a4, a5, a6) # a1 ~ a6를 제1행에 3개, 제2행에 3개로 2x3 행렬로 reshape

2×3 Array{Int64,2}:
 1  2  3
 4  5  6

In [43]:
[a1 a2; a3 a4; a5 a6] # 3x2 행렬 만들기

3×2 Array{Int64,2}:
 1  2
 3  4
 5  6

In [44]:
# 이 코드는 바로 위가 Parsing된 결과이다. 즉, 3x2 행렬을 만든다. (각 행에 2개씩)
hvcat((2, 2, 2), a1, a2, a3, a4, a5, a6)

3×2 Array{Int64,2}:
 1  2
 3  4
 5  6

* 만약 `blocks_per_row_as_tuple`이 `tuple`이 아닌 Single Integer이면, 모든 행에 대하여 각 행에 그 숫자만큼 원소들이 들어간다고 보고, 자동으로 행렬을 만든다.

In [45]:
hvcat(2, a1, a2, a3, a4, a5, a6) # 각 행에 2개씩 들어가므로 3x2 행렬이 나옴.

3×2 Array{Int64,2}:
 1  2
 3  4
 5  6

### adjoint (`Base.adjoint`): `arr'`

* **Usage**: `adjoint(Matrix or Array)`
* **Desc**: 주어진 배열들 또는 행렬의 *수반 행렬 (Adjoint Matrix)* 을 구한다. 복소행렬의 경우 수반행렬은 그 Transpose에 각 원소에 켤례를 취한 것임에 주의. (즉, Conjugate Transposition 행렬을 구한다 -> Julia에서는 행렬 뒤에 연산자 `'`를 사용하면 그 행렬의 Conjugate Transposition을 수행한다. 즉, `arr'`은 자동으로 Julia에 의해 `adjoint(arr)`로 Parsing된다.

In [46]:
A = [3+2im 9+2im; 0 0] # 2 x 2 complex matrix

2×2 Array{Complex{Int64},2}:
 3+2im  9+2im
 0+0im  0+0im

In [47]:
B = A' # <=> B = adjoint(A)

2×2 LinearAlgebra.Adjoint{Complex{Int64},Array{Complex{Int64},2}}:
 3-2im  0+0im
 9-2im  0+0im

* 당연히 Real Matrix들에 대해서는, `adjoint` Operation은 그 전치행렬(Transpose)을 구한 것과 효과가 동일하다.

In [49]:
A = [1 2; 3 4]
A' # will be parsed as adjoint(A), but since matrix a is a real-matrix, therefore the result will be same with transpose(A)

2×2 LinearAlgebra.Adjoint{Int64,Array{Int64,2}}:
 1  3
 2  4

### getindex (`Base.getindex`): `arr[idx]`

* **Usage**: `getindex(collection, index_or_key...)`
* **Desc**: 주어진 Collection object에서, 제시한 index 또는 key들에 상응하는 값들을 돌려준다. Collection Indexing Operator `[]`는 자동으로 이 `getindex`로 Parsing된다. 즉, `arr[idx]`는 Julia에서 `getindex(arr, idx)`로 Parsing된다.

In [50]:
# Plain Array Indexing
arr = [1, 2, 3, 4]
println(arr[1]) # <=> getindex(arr, 1)
println(getindex(arr, 1))

1
1


In [51]:
# Plain Matrix Indexing
mat = [1 2 3; 4 5 6]
println(mat[1,2]) # <=> getindex(mat, 1, 2)
println(getindex(mat, 1, 2))

2
2


In [52]:
# Dictionary Indexing
dic = Dict("a" => 1, "b" => 2)

Dict{String,Int64} with 2 entries:
  "b" => 2
  "a" => 1

In [53]:
println(getindex(dic, "a")) # get the value with key "a", from the Dictionary dic
println(dic["a"]) # Python-like Indexing for Dictionary.

1
1


### setindex! (`Base.setindex!`): `arr[idx] = val; val`

* **Usage**: `setindex!(collection, value, key_or_indexes...)`
* **Desc**: 주어진 Collection Object에서, 제시한 index 또는 key들에 상응하는 공간을 제시한 value로 set한다. 즉 `arr[idx] = val`은 Julia에서 `setindex!(arr, val, idx); val`로 Parsing된다.
* **Note**: Julia는 Function의 Naming Convection상, 이름의 끝이 !라면 내부에서 넘겨준 인자의 Modification이 일어난다는 의미임을 상기하자.

In [54]:
# Plain Array Indexing
arr = [1, 2, 3, 4]
arr[1] = 3 # <=> setindex!(arr, 3, 1)
println(arr)

arr = [1, 2, 3, 4]
setindex!(arr, 3, 1)
println(arr)

[3, 2, 3, 4]
[3, 2, 3, 4]


In [55]:
# Plain Matrix Indexing
mat = [1 2 3; 4 5 6]
mat[1, 2] = 10 # <=> setindex!(mat, 10, 1, 2)
println(mat)

mat = [1 2 3; 4 5 6]
setindex!(mat, 10, 1, 2)
println(mat)


[1 10 3; 4 5 6]
[1 10 3; 4 5 6]


In [57]:
# Dictionary Indexing
dic = Dict("a" => 1)
dic["a"] = 100 # <=> setindex!(dic, 100, "a")
println(dic)

dic = Dict("a" => 1)
setindex!(dic, 100, "a")
println(dic)

Dict("a" => 100)
Dict("a" => 100)


### getproperty (`Base.getproperty`): `A.n`

* **Usage**: `getproperty(value, name::Symbol, (Optional) order::Symbol)`
* **Desc**: 주어진 `value`에서 `name`에 해당하는 Property를 return한다. 간접 연산자 `.`가 이 함수로 Parsing되는데, `a.b`는 `getproperty(a, :b)`를 Call한다.

In [58]:
struct MyType
    x
end # Julia는 구조체를 지원하는 것 같다.

function Base.getproperty(obj::MyType, sym::Symbol) # Base.getproperty에 대해 function overloading.
    # 만약 value가 MyType 또는 그 Sub-type이면 여기서 정의한 Overloading된 함수 Body가 실행될 것.
    if sym === :special
        return obj.x + 1 # <=> return getproperty(obj, :x) + 1
    else
        # fallback to getfield
        return getfield(obj, sym)
    end
end

obj = MyType(1)
println(obj.special)
println(obj.x)

2
1


### setproperty! (`Base.setproperty!`): `A.n = x`

* **Usage**: `setproperty!(value, name::Symbol, x, order::Symbol)`
* **Desc**: 주어진 `value`에서 `name`에 해당하는 Property를 `x`로 설정한다. 간접 연산자 `.`를 사용하여 할당 연산을 하는 경우, 즉 `A.n = x`는 Julia가 `setproperty!(A, :n, x); x`로 Parsing한다.

* Module에 대한 `setproperty!`의 Call은 Julia 1.8부터 가능하다. 여기 설치된 Julia는 1.4.1이므로 불능.

In [69]:
mutable struct MyMutableType # 기본적으로 Julia에서 struct 구조체는 Immutable이다. 수정 가능하게 하려면 mutable keyword를 사용해야 함.
    x
end

obj = MyMutableType(0)
obj.x = 1 # <=> setproperty!(obj, :x, 1)
println(obj.x)
obj.x = 0
setproperty!(obj, :x, 1)
println(obj.x)

1
1


## Anonymous Functions (익명 함수)

* Julia에서 함수는 [First-class Object](https://en.wikipedia.org/wiki/First-class_citizen)들이다. 즉, 다른 여타 Entity들에 대해서도 지원하는 연산들을 모두 지원하는 Entity이다. 따라서 함수를 함수의 인자로 쓸 수도 있고, 함수의 return값으로 함수를 돌려줄 수도 있으며, '익명 함수'를 Python과 마찬가지로 만들 수 있다.

* **Julia에서는 함수를 정의할 때 함수의 이름을 생략하여 익명 함수를 만들 수 있다.** 함수를 통상적 문법으로 정의하는 경우에는 `function` 예약어와 Argument List 사이에 이름을 생략하면 되고, 축약형 문법으로 정의하는 경우에는 대입 연산자가 아니라 매개변수 list `->` return값의 형태로 정의할 수 있다.

In [70]:
# 축약형 문법으로 정의하는 익명 함수
x -> x^2 + 2x - 1

#3 (generic function with 1 method)

In [71]:
function(x)
    x^2 + 2x - 1
end

#5 (generic function with 1 method)

* Julia에서 익명 함수를 정의하면 'Identifier'는 없지만, Julia Compiler가 자체적으로 번호를 부여, 이름으로 사용하여 (#N 식의 이름을 부여) Generic Function으로 자동으로 정의됨에 주의.

* Python에서와 동일하게, 익명 함수들은 보통 다른 함수에 인자로 간단한 함수를 넘기는 경우에 종종 사용한다. 대표적인 예시가 `map` 함수인데, Python에서의 `map` 내장함수와 마찬가지로 Julia의 `Base.map(function, collection...) -> collection`은 넘겨준 collection의 각 원소에 function을 적용한 결과값을 각각의 원소 자리에 대체시킨 collection을 return해준다. 다만 Python의 경우 `map`의 return type이 `map` object여서 Casting이 필요했지만, Julia의 경우 `map`은 넘겨준 Collection과 동일 Type이 return된다.

In [72]:
map(round, [1.2, 3.5, 1.7]) # 1차원 3개 원소 array에 각각 round 함수를 적용한 결과 array를 return

3-element Array{Float64,1}:
 1.0
 4.0
 2.0

In [73]:
# 위와 같이 사전에 정의된 함수가 아니라, 익명 함수를 인자로 넘겨주어 map을 사용할 수 있다.
map(x -> x^2 + 2x - 1, [1, 3, -1])

3-element Array{Int64,1}:
  2
 14
 -2

* 익명함수를 축약형으로 정의하는 경우, 다인자 익명함수를 정의하고자 할 때에는 Arg List 자리를 순서대로 Tuple의 형태로 열거해주면 된다: E.g) `(arg1, arg2, arg3) -> return_val`
* 만약 익명함수를 축약형으로 정의할 때, 무인자 익명함수를 정의하고자 할 때는 Arg List를 비우면 된다: E.g) `() -> return_val`

## Tuples

* Python과 유사하게, Julia에도 Built-in Data Structure로 `Tuple`이 있다. Tuple은 Python에서와 마찬가지로, 고정 길이 Container 자료형이고, Immutable이며 다른 Type의 자료를 동시에 들고 있을 수 있다.
* Tuple의 구성법은 Python의 문법과 정확히 동일하다.

In [74]:
typeof((1, 1+1))

Tuple{Int64,Int64}

In [75]:
(1,)

(1,)

In [76]:
x = (0.0, "hello", 6*7) # just like in Python, Tuple can hold multiple type at once.

(0.0, "hello", 42)

In [77]:
x[2] # Just like python, Tuple can be indexed. But caution, 'cause indicies in Julia start from 1., not 0.

"hello"

## Named Tuples

* Python과는 다르게, Julia는 `NamedTuple`이라는 자료형을 추가로 가지는데, 이는 Tuple의 각 자리의 값에 상응하는 이름(Identifier)를 줄 수 있다. 이는 Tuple의 원소를 열거할 때 각 자리에 `identifier=value`의 형태로 명시해주면 된다.
* 이들 Tuple의 각 자리의 이름은 Tuple의 property(field)처럼 작동하여, 간접연산자 `.` 또는 `getproperty()` 내장함수로 그 값을 얻을 수 있다.

In [78]:
x = (a=2, b=1+2) # Constructing Named Tuples in Julia

(a = 2, b = 3)

In [79]:
typeof(x)

NamedTuple{(:a, :b),Tuple{Int64,Int64}}

In [80]:
x[1] # NamedTuple can be indexed with traditional indexing operator []

2

In [81]:
x.a # NamedTuple can be indexed, with dot accesing operator, just like accessing any property in an object.

2

In [83]:
getproperty(x, :a) # <=> x.a

2

## Packing & Unpacking in Julia

### Normal Packing & Unpacking (Destructuring)

* Python에서의 Packing과 Unpacking을 Julia에서도 동일하게 할 수 있다.

In [84]:
# 1:3은 Matlab에서 그러하듯, 1부터 3까지의 값을 순차대로 generating하는 iterator이다.
(a, b, c) = 1:3

1:3

In [85]:
b

2

* Python에서와 마찬가지로, Julia에서 function이 여러 개의 return value를 넘길 때 Packing을 이용하여 return value들을 합친 tuple을 넘길 수 있고, 외부에서는 이 tuple을 받아서 unpacking하여 사용할 수 있다.

In [86]:
function foo(a, b)
    a+b, a*b # packing
end

foo(2,3) # returning packed tuple

(5, 6)

In [87]:
# unpacking available in Julia, just like Python
x , y = foo(2, 3)

(5, 6)

In [88]:
println(x); println(y)

5
6


* Python과 마찬가지로 이 Packing & Unpacking 기능을 이용하여 별도의 변수 없이 Swapping을 구현할 수 있다.

In [89]:
x, y = foo(2,3); y, x = x, y; println(x); println(y)

6
5


* Unpacking을 할 때 필요없는 값의 경우, `_`로 필요없는 값을 저장할 Identifier를 명명하면 자동으로 Julia가 Unpacking 이후 그 값들은 즉시 버려버린다. **(`_`는 Invalid Identifier이지만, 이 경우에만 오직 예외적으로 허용된다)**

In [90]:
_, _, _, d = 1:10

1:10

In [91]:
d

4

In [92]:
# Example
X = zeros(3)

X[1], (a,b) = (1, (2, 3)) # X[1] = 1, (a, b) = (2, 3) and unpacking again.
println(X)
println(a); println(b)

[1.0, 0.0, 0.0]
2
3


* **Julia 1.6 이후부터는** Unpacking을 할 때 마지막 변수 뒤에 `...` (Sluping)을 붙이면 (공백 없이) Unpacking의 자리가 충분하지 않은 경우 뒤쪽 끝까지 마지막 변수에 집어넣어준다.

In [1]:
a, b... = "hello"

"hello"

In [2]:
println(a); println(b)

h
ello


In [1]:
# Another Example
a, b... = Iterators.map(abs2, 1:4) # 1:4 Iterator에 mapping을 적용 -> Generator가 나옴.

Base.Generator{UnitRange{Int64}, typeof(abs2)}(abs2, 1:4)

In [2]:
a

1

In [3]:
b

Base.Iterators.Rest{Base.Generator{UnitRange{Int64}, typeof(abs2)}, Int64}(Base.Generator{UnitRange{Int64}, typeof(abs2)}(abs2, 1:4), 1)

* **Julia 1.9 이후부터는** Unpacking을 할 때 마지막 변수가 아니더라도, 중간 변수에도 `...` (Sluping)을 붙이면 딱 맞게 Unpacking의 개수가 맞지 않더라도 Sluping한 Variable에 최대한 많이 넣어 준다.

In [4]:
# Availabel from Julia v1.9
a, b..., c = 1:5 # a -> 1, b -> 2:4, c -> 5

1:5

In [5]:
a

1

In [6]:
b

3-element Vector{Int64}:
 2
 3
 4

In [7]:
c

5

In [8]:
# Another Example
front..., tail = "Hi!"

"Hi!"

In [9]:
println(front); println(tail)

Hi
!


* 한편, 뒤에 소개하겠지만 함수에서 가변 인자를 사용하는 경우, `...` (Slurping) 은 반드시 가장 뒤 Parameter에만 쓸 수 있다. 그러나 인자를 1개 Argument로 받아서 Unpacking(Destructuring)하는 경우에는 적용되지 않고, 중간에 써도 큰 문제가 없다.

In [10]:
f(x..., y) = x # Invalid Function Definition, since Slurping is not located at the end of the parameter list of the function.

LoadError: syntax: invalid "..." on non-final argument around In[10]:1

In [11]:
f((x..., y)) = x # Valid Function Definition,
# since function f defined in here, is specifying only one argument - a tuple but that might have variaent length.

f (generic function with 1 method)

In [13]:
f((1, 2, 3)) # x -> (1, 2), y -> 3, returns x -> therefore (1, 2)

(1, 2)

### Property Unpacking (Destructuring)

* Python과 달리 Julia는 `NamedTuple`이 있는데, 이 NamedTuple의 이름과 같은 Identifier들로 Unpacking을 하면, 그 Identifier에 맞게 Unpacking 된다.

In [14]:
b, a, c = (a=1, b=2, c=3)

(a = 1, b = 2, c = 3)

In [15]:
println(a); println(b); println(c)

2
1
3


### Argument Destructuring

* Python과는 달리 Julia는 Function을 정의할 때, Function Parameter List 자체에서 Unpacking을 수행할 수 있다. 즉, 다인자 함수가 아닌 Tuple의 형태로 Parameter를 정의하더라도, 자동으로 딱 맞게 Unpacking이 일어난다. 

In [16]:
# minmax의 return값은 Tuple임에 주의.
minmax(x, y) = (y < x) ? (y, x) : (x, y)
gap((min, max)) = max - min # gap 함수의 Parameter List에서 Unpacking을 사용.
gap(minmax(10, 2)) # minmax(10, 2)의 반환값 (2, 10)이 gap의 parameter list에서 첫 번째 match인 (min, max) tuple에서 Unpacking 된다.

8

In [19]:
# NamedTuple이나 Structure 등에 정의된 Properties에 대한 Unpacking도 당연히 Argument 안에서 가능.
foo((; x,y)) = x + y
foo((x=1, y=2)) # Passing NamedTuple

3

In [20]:
struct A
    x
    y
end
foo(A(3,4)) # Passing a structure

7

* **익명 함수에서 Argument Destructuring을 사용하는 경우, 끝에 Comma(`,`) 한 개를 더 찍어주어야 한다.**

In [21]:
# map 내장함수에 익명함수를 넘겨주는데, 이 익명함수는 Container의 Iterator의 Point OBj를 받아서 Unpacking한뒤 그 합으로 대체.
# The following is invalid:
map(((x, y)) -> x + y, [(1,2), (3,4)])

LoadError: MethodError: no method matching (::var"#3#4")(::Tuple{Int64, Int64})

[0mClosest candidates are:
[0m  (::var"#3#4")(::Any, [91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[21]:3[24m[39m


In [22]:
# The following is valid (need additional trailing comma for argument unpacking in lambda functions
map(((x,y),) -> x + y, [(1,2), (3,4)])

2-element Vector{Int64}:
 3
 7

## Varargs Functions (가변인자 함수)

> **Varargs Functions (가변인자 함수)**
>
> 임의 개수의 인자(Aribitary Number of Arguments)를 받는 함수를 **가변인자 함수 (Varargs Functions)**라 한다. (*Stands for Variable Number of Arguments*)

* Julia에서는 함수를 정의할 때, 함수의 Parameter List 마지막에 `...` (Suprling) 을 공백 없이 달아주면 마지막 Identifier의 Parameter가 가변 인자 Parameter가 되어, 그 이외의 Parameter에 인자를 차례로 넣은 뒤 **남은 인자들을 Tuple의 형태로 받아오게 된다.**

In [23]:
# Varargs Functions Example
bar(a, b, x...) = (a, b, x) # This function has a Variable Args, and its identifier is x

bar (generic function with 1 method)

In [24]:
bar(1,2) # x will be empty tuple

(1, 2, ())

In [25]:
bar(1, 2, 3) # x is tuple, but only has 3

(1, 2, (3,))

In [26]:
bar(1, 2, 3, 4) # x is tuple, but has 3 and 4

(1, 2, (3, 4))

In [27]:
bar(1, 2, 3, 4 ,5, 6) # x is tuple, but has 3, 4, 5, 6

(1, 2, (3, 4, 5, 6))

* Variable Argument에 들어올 수 있는 가변 인자의 개수를 제한하는 방법도 있지만, 이는 [Parametrically-constrained Varargs methods](https://docs.julialang.org/en/v1/manual/methods/#Parametrically-constrained-Varargs-methods)에서 다룸.

## Iteratable Object Splatting

* Python의 Asterisk(`*`)를 이용한 Tuple Splatting과 같이, **Julia에서도 Container 자료형 (정확히는 Iteratable 자료형) 뒤에 `...` (Suprling) 을 공백 없이 달아서 이들을 Splatting 할 수 있다.**

In [29]:
x = (3,4)
bar(1, 2, x...) # <=> bar(1, 2, 3, 4)
# same as bar(1, 2, *x) in python

(1, 2, (3, 4))

In [30]:
x = (2, 3, 4)
bar(1, x...) # <=> bar(1, 2, 3, 4)

(1, 2, (3, 4))

In [31]:
x = (1, 2, 3, 4)
bar(x...) # <=> bar(1, 2, 3, 4)

(1, 2, (3, 4))

In [32]:
# Just like python, splatting can be applied into any ITERATABLE OBJECT
x = [3, 4] # Vector is also iteratable object.
bar(1, 2, x...)

(1, 2, (3, 4))

In [33]:
x = [1, 2, 3, 4]
bar(x...)

(1, 2, (3, 4))

* 아주 당연하게도, Iteratable Object의 Splatting은 굳이 가변인자 함수가 아닌 임의의 Function Call에서도 가능하다.

In [34]:
baz(a, b) = a + b
args = [1, 2]

2-element Vector{Int64}:
 1
 2

In [35]:
baz(args...) # Note that function 'baz' is not varargs function

3

In [36]:
args = [1, 2, 3]
baz(args...) # Note that the function 'baz' does not accept three args, therefore this will yield an error

LoadError: MethodError: no method matching baz(::Int64, ::Int64, ::Int64)

[0mClosest candidates are:
[0m  baz(::Any, ::Any)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[34]:1[24m[39m


## Optional Arguments (Default Value for Argument)

* Julia에서도 Python과 마찬가지로 Parameter List를 열거할 때, Identifier 뒤에 `=value`의 형태로 Default Value를 지정하여, 해당 Argument를 Optional Argument로 지정할 수 있다.
* Python, C++와 정확히 같은 이유로 이러한 Optional Argument는 함수 Parameter List의 뒤쪽에 모두 연속되어 있어야 한다. (앞에 이러한 Optional Argument를 허용하면, Optional Argument가 생략된 것인지 아니면 생략되지 않은 것인지 알기 어려워진다)

In [40]:
using Dates
function date(y::Int64, m::Int64=1, d::Int64=1) # Set default value for parameter m and d to 1
    err = Dates.validargs(Date, y, m, d)
    err === nothing || throw(err) # Short-circuit evaluation을 이용한 short-handed if conditional
    return Date(Dates.UTD(Dates.totaldays(y,m,d)))
end
# 이상의 함수는 Date function에서 UTInstant{Day}를 만들어줌.

date (generic function with 3 methods)

In [41]:
date(2000, 12, 12)

2000-12-12

In [42]:
date(2000, 12) # d omitted, therefore using 1, specified as the default value of d

2000-12-01

In [43]:
date(2000) # m and d omitted, therefore using 1, each specifed as the default values of m and d

2000-01-01

In [44]:
methods(date)

# 3 methods for generic function "date" from Main:
 [1] date(y::Int64)
     @ In[40]:2
 [2] date(y::Int64, m::Int64)
     @ In[40]:2
 [3] date(y::Int64, m::Int64, d::Int64)
     @ In[40]:2

* Optional Argument를 활용하면 굳이 Function Overloading 없이도 다른 개수의 인자를 받는 같은 이름의 함수를 쉽게 정의할 수 있다.

## Keyword Arguments

* Python과 마찬가지로, Julia에서도 Function에 대한 Keyword Argument를 지원한다.
* 그러나 Python의 경우는 일반적인 Argument와 구분 없이, 곧바로 keyword argument들은 `key=value`의 형태와 같이 Parameter List 안에 열거해주면 되었지만, Julia의 경우는 일반 Argument (Vaargs Argument 포함)와 Keyword Argument들을 구분한다.
* Julia Function에서는 Parameter List에서 반드시 Keyword Arguments들이 가장 뒤에 연속적으로 열거되어 있어야 하며, 열거의 형식은 다른 종류의 Argument를 모두 열거한 뒤, **세미콜론 (`;`)** 을 쓴 뒤 Python에서 하는 것처럼 Keyword Argument들을 열거해주는 것이다.

In [46]:
# Example of Keyword Arguments
function plot(x, y; style="solid", width=1, color="black")
    ### Note that after a semi-colon(;), the keyword arguments are listed.
end

plot (generic function with 1 method)

* 단, Keyword Argument들은 Function Call에서는 Python과 같은 형태로 Call이 가능하다. 즉, Julia의 경우 Function Definition에서만 세미콜론(`;`)이 Keyword Argument들을 구분하는데 필요하지, Function Call에서는 불필요하다.
* 그러나 Function Definition Style과 마찬가지의 Style로 Function Call에서도 세미콜론(`;`)으로 Keyword Argument들을 구분해주어도 된다.
  

In [47]:
plot(1, 2, style="solid")

In [48]:
plot(1, 2; style="solid") # The keyword argument seperator, ; is optional for function call in Julia.

* Keyword Argument는 Varargs Functions에도 사용 가능하고, Python에서의 `**kargs`와 마찬가지로, 추가적인 Keyword Argument도 `...` (Suprling)을 keyword arg identifier 뒤에 공백 없이 붙여 받을 수 있다.

In [50]:
function func1(x...; style="solid")
    ###
end

func1 (generic function with 1 method)

In [51]:
function func2(x; y=0, kargs...) # Note that y is keyword argument in here.
    ###
end

func2 (generic function with 1 method)

* 함수 내부에서 Keyword Argument는 **Immutable Key-Value Iterator over a named tuple**로 저장된다.
* NamedTuple도 Keyword Arguments로, Function Call에서 Unpacking되어 넣어줄 수 있다.

In [54]:
function func3(x; arg1=0, arg2=0, arg3=0)
    println(x)
    println(arg1 + arg2 + arg3)
    return
end

keywords = (arg1=1, arg2=2, arg3=3) # NamedTuple
func3(1; keywords...) # Unpacking Available for NamedTuple, passing key-value pairs as normal keyword arguments.

1
6


* 다만 위와 같이 Keyword Arguments Unpacking을 사용하는 경우, 세미콜론(`;`)을 사용하여 일반 Argument와 구분해주어야만 `keywords...`와 같이 Unpacking이 일반적인 Arguments Unpacking이 아닌 Keyword Arguments Unpacking으로 처리됨에 주의.

* **Julia에서는 Keyword Argument에 대한 Default Value를 지정하지 않는 것이 허용된다.** 이 때, Default Value가 없는 Keyword Argument에 대해 함수 Call에서 그 값을 넘기지 않는 경우, `UndefKeywordError`가 raise된다.

In [55]:
function f(x;y) # y is keyword argument here.
    ###
end

f(3, y=5) # ok, y is assigned
f(3) # throws UndefKeywordError(:y), since y is non-default valued keyword argument

LoadError: UndefKeywordError: keyword argument `y` not assigned

* Julia의 경우 Meta Programming을 지원하기 때문에, Function Call에서 Keyword Argument를 넘길 때 `key=value`의 형태가 아니라 `:key => value`의 형태로 넘겨도 된다. 이 때 `:` Operator가 뒤에 오는 변수의 값 자체를 Symbol로 해석하라는 의미를 가지고 있기 때문에, keyword 이름이 runtime에 변할 수 있는 경우 그 이름을 변수에 담아두고 Symbol로 만들어서(`:`) keyword argument를 넘겨줄 수 있다.

In [59]:
function plot(x, y; style="solid", width=1, color="black")
    ### Note that after a semi-colon(;), the keyword arguments are listed.
end

plot(1, 2; :width => 2) # <=> equivalent to plot(x, y, width=2)

* Keyword Argument는 Argument를 Key들로 구분하므로, 일반적인 Argument들과는 달리 중간에서 Unpacking하여 Call이 가능하다.

In [61]:
function plot(x, y; style="solid", width=1, color="black")
    ### Note that after a semi-colon(;), the keyword arguments are listed.
end
options = (style="solid", color="black")
plot(1, 2; options..., width=1)

* 위와 같이 중간에서 Kargs Unpacking으로 Function Call을 하는 경우, Unpacking한 `options...`에 `width`가 명시되어 있더라도, **가장 오른쪽 (Right-most)** 의 karg가 우선 적용된다.
* 그러나 명시적으로 Function Call에서 동일 Identifier를 가지는 Keyword Argument를 중복 명시하는 경우는 허용되지 않는다. (Syntax Error)

In [64]:
function plot(x, y; style="solid", width=1, color="black")
    ### Note that after a semi-colon(;), the keyword arguments are listed.
    print("width: ")
    println(width)
end

options = (style="solid", width=1, color="black")
plot(1, 2; options..., width=3) # right most will be applied firstly.

width: 3


In [65]:
plot(1, 2; width=3, width=1) # Not Allowed

LoadError: syntax: keyword argument "width" repeated in call to "plot" around In[65]:1

## Function Default Value의 Evaluation Scope

* Julia에서 Function Parameter List에서, 각각의 Parameter는 열거된 이후 (오른쪽과 아래) 에서 Known하다. (The scope of the function parameter, is from the just right after it is specified to the end of the function.)

In [70]:
b = 5
function func_def(x; a=b, b=1) # b is unknown, when evaluating parameter list a=b, b does not refering to b=1, but b in outer scope.
    println(a)
end
func_def(1)

5


## Do-Block Syntax for Function Arguments

* 함수를 Argument로 넘겨야 하는 함수의 경우, 함수가 간단하지 않은 함수일 경우 익명 함수의 형태로 이러한 함수를 인자로 넘기면 코드가 복잡해져 가독성이 떨어지는 문제가 있었다. 이를테면:

In [73]:
# 복잡한 함수를 map 함수를 이용해 1차원 Array에 적용하는 경우
map(x -> begin
        if x < 0 && iseven(x)
            return 0
        elseif x == 0
            return 1
        else
            return x
        end
    end,
    [-2, 0, 1]) # 너무 복잡하다.

3-element Vector{Int64}:
 0
 1
 1

* 따라서 Julia에서는 `do` 키워드를 이용해서, 함수가 '함수를 인자로 받는 경우', **인자로 넘겨줄 함수를 `do` 키워드 뒤로, 즉 함수 Call 밖으로 빼 주는 기능을 제공한다. 정확하게는 `do` 키워드 뒤에 익명 함수의 Parameter와 그 정의를 써 주고 `end`로 닫으면, 익명 함수를 만들어서 `do` 키워드 앞의 함수의 첫 번째 인자로 넘겨준다.**
    * 즉, Function f에 대해 `f(x -> begin ... end)`는 `f do ...`와 같다.

In [74]:
# 상기 예시는 다음과 정확히 같다. do-block 문법을 사용하면 anonymous function을 조금 더 가독성 좋게 넘겨줄 수 있다.
map([-2, 0, 1]) do x # x specified here, acts as the parameter of the anonymous function
    if x < 0 && iseven(x)
        return 0
    elseif x == 0
        return 1
    else
        return x
    end
end

3-element Vector{Int64}:
 0
 1
 1

* 즉, 상기 예제에서 `do x ...` Syntax는 `x`를 Parameter로 하는 익명 함수를 만들고, map 내장함수의 첫 번째 인자로 던져준다.

* `do-block`을 사용하여 익명함수를 전달하고자 하는 경우, 익명함수를 다인자 함수로 하고자 한다면 `do (a,b)`와 같이 `do` 키워드 다음에 인자들을 열거한 Tuple을 써 주면 된다.
* `do-block`을 사용하여 익명함수를 전달할 때, 인자가 없는 익명함수를 만들어 넣어주고 싶다면 `do` 키워드만 쓰면 된다. (즉, Single `do`는 `() -> ...` 꼴의 익명함수와 같은 효과를 낸다)

* 이와 같은 Julia에서의 Function Application의 `do-block` Syntax는 함수 안에 익명 함수의 복잡한 정의를 넣는 것보다도 가독성이 좋으므로 다음과 같이 구체적으로 활용될 수 있다.

In [ ]:
open("outfile", "w") do io
    write(io, data)
end
# <=> open(io -> write(io, data), "outfile", "w")

* 이상의 코드는 `open` 내장함수의 여러 Overloading Version 중 하나가 다음과 같은 정의를 가지기 때문에 자연스럽게 실행된다.

```julia
function open(f::Function, args...)
    io = open(args...)
    try
        f(io)
    finally
        close(io)
    end
end
```

## Function Composition (함수 합성) & Piping (함수 연쇄 실행)

* Julia에서는 함수를 합성하거나 연쇄적 실행이 가능하다.

### 함수 합성 (Function Composition)

* Julia에서는 함수 합성 Operator (Function Composition Operator) `∘`를 사용하여 두 함수를 합성할 수 있다. (통상적인 수학에서의 함수 합성 기호로!)
    * 이 기호는 Julia REPL에서 `\circ` + `Tab`으로 입력이 가능하다.
* **E.g.)** `(f ∘ g)(args...)` <=> `f(g(args...))`

In [75]:
# Example
(sqrt ∘ +)(3,6) # <=> sqrt(+(3,6)) <=> sqrt(3 + 6)

3.0

In [76]:
# Another Example, using map
map(first ∘ reverse ∘ uppercase, split("you can compile functions like this"))
# 문자열을 공백 기준으로 분리, 모두 대문자화(uppercase) -> 뒤집고(reverse) -> 그 첫글자 가져오기(first)

6-element Vector{Char}:
 'U': ASCII/Unicode U+0055 (category Lu: Letter, uppercase)
 'N': ASCII/Unicode U+004E (category Lu: Letter, uppercase)
 'E': ASCII/Unicode U+0045 (category Lu: Letter, uppercase)
 'S': ASCII/Unicode U+0053 (category Lu: Letter, uppercase)
 'E': ASCII/Unicode U+0045 (category Lu: Letter, uppercase)
 'S': ASCII/Unicode U+0053 (category Lu: Letter, uppercase)

### 함수 연쇄 실행 (Function Chaining or Piping)

* Julia에서는 함수 Chaining Operator (Piping Operator) `|>` 를 이용하여 함수의 출력을 다른 함수의 입력으로 이어줄 수 있다.
    * **E.g.)** `x |> f |> g` <=> `g(f(x))`

* 함수의 합성은 합성 연산자 뒤에 오는 함수를 먼저 적용하지만, 함수의 연쇄 실행은 연산자 뒤에 오는 함수를 나중에 적용하는 차이가 있음에 주의하자.

In [77]:
# Example
1:10 |> sum |> sqrt # <=> sqrt(sum(1:10))

7.416198487095663

* Pipe Operator는 Element-wise하게 적용될 수도 있다. 즉, `.|>` 연산이 지원된다.

In [78]:
# Example for Vectorized Chaining
["a", "list", "of", "strings"] .|> [uppercase, reverse, titlecase, length]
# <=>
# [uppsercase("a"), reverse("list"), titlecase("of"), length("strings")]

4-element Vector{Any}:
  "A"
  "tsil"
  "Of"
 7

* Piping Operator와 익명 함수를 동시에 사용할 때는 익명함수와 Piping을 구분하기 위해 괄호를 쳐 주어서 구분되게 해야 한다.

In [79]:
1:3 .|> (x -> x^2) |> sum |> sqrt
# sqrt(sum((1:3) .^ 2))

3.7416573867739413

In [80]:
1:3 .|> x -> x^2 |> sum |> sqrt
# <=> 1:3 .|> (x -> x^2 |> sum \> sqrt)

3-element Vector{Float64}:
 1.0
 2.0
 3.0

## Dot Syntax for Vectorizing Functions

* 많은 Data Science 언어에서, for-loop에서 일일이 Function을 call하는 것보다는 Function을 Vectorized Version으로 만들어 Container 각각의 Value에 대해 전체적으로 빠르게 적용해주는 것이 성능 면에서 훨씬 좋다. Julia는 이러한 Function Vectorization을 별도의 함수 호출 없이, '연산자'를 사용하여 곧바로 할 수 있도록 해준다.

* **Julia는 임의의 함수를 선언한 경우, 그 함수 이름 뒤에 곧바로 Dot Operator `.`를 붙여 그 함수를 Vectorization할 수 있다.**

In [82]:
arr = [1.0, 2.0, 3.0]
sin.(arr)

3-element Vector{Float64}:
 0.8414709848078965
 0.9092974268256817
 0.1411200080598672

* 물론 함수 자체를 Vectorizative하게 선언해서, 이를테면 `f(A::AbstractArray) = map(f, A)`와 같이 정의한 경우는 `f(A)`로 곧바로 실행이 가능하고, 이 역시 `f.(A)`만큼이나 효율적이다. 그러나 Julia는 이러한 Dot Operator를 통한 Function Vectorization을 지원하기 때문에, 함수 설계 단계에서부터 Vectorization을 크게 고려하지 않아도 된다는 장점이 있다.

* **조금 더 정확하게는,** Julia는 `f.(args...)`를 `broadcast(f, args...)`로 Parsing하여 처리한다. 즉, Julia의 Dot Operator를 통한 Function Vectorization은 Broadcasting까지 알아서 해 준다.

In [83]:
# Example
f(x, y) = 3x + 4y
f.(pi, [1, 2, 3]) # pi - a single value will be expanded to [pi, pi, pi] during the broadcasting

3-element Vector{Float64}:
 13.42477796076938
 17.42477796076938
 21.42477796076938

In [85]:
# Another Example
hap(x1, x2) = x1 + x2
hap.([1, 2, 3], [4, 5, 6]) # returns [1 + 4, 2 + 5, 3 + 6]

3-element Vector{Int64}:
 5
 7
 9

* Dot Operator를 통한 Function Vectorization에서, Keyword Argument들은 Broadcasting되지는 않는다. 대신, Each Function Call에서 Passing된다.
    * **E.g.)** `round.(x, digits=3)` <=> `broadcast(x -> round(x, digits=3), x)`

* Julia는 Performance를 위해서, Nested Dot Operator를 통한 Function Vectorization Call은 Single Broadcast Loop으로 합쳐서 해 준다.
* 즉, 이를테면 `sin.(cos.(X))`는 `broadcast(x -> sin(cos(x)), X)` 또는 `[sin(cos(x)) for x in X]`와 같다.
* 이러한 Nested Dot Operator에서 Single Broadcast Loop으로의 Fusion은 Compiler 수준에서 최적화를 위해 할 수도 있고 하지 않을 수도 있는 것이 아니다. 반드시 일어난다. (*Syntactic Guarantee*)
* 기술적으로 Nested Function Call에서 Dot Fusion은 Non-Dot Function을 만나면 거기서 종료된다. 이를테면, `sin.(sort(cos.(X)))`의 경우 sin 함수 계산용 loop와 cos 함수 계산용 loop는 sort 함수 때문에 fusion되지 않는다.

* 성능을 위해서는 보통 Output Array를 미리 Allocate해 두는 경우가 Python에서 종종 있는데, Julia에서는 `X .= ...`와 같은 형태로 미리 Allocate한 Array에 대해 Element-wise하게 계산이 가능하다. `.=` 또한 Nested Loop에 자동으로 Fusion된다.
    * **E.g.)** `X .= ...` <=> `broadcast!(identity, X, ..._` (Except, the `broadcast!` loop is fused with any nested "dot" calls.)
    * **E.g.)** `X .= sin.(Y)` <=> `broadcast!(sin, X, Y)` (Overwriting `X` with `sin.(Y)` IN-PLACE)
* 이러한 Pre-Allocated Array에 대한 Fusion은 Element-wise 대입 연산자 `.=`의 좌변이 Array-Indexing Expr인 경우에도 일어난다.
    * 이를테면, `X[begin+1:end] .= sin.(Y)`의 경우 `broadcast!(sin, view(X, firstindex(X)+1:lastindex(X)), Y)`로 자동으로 Parsing되어 In-place로 X가 Update된다.

### Macro `@.`를 이용한 전체 식 Vectorization

* 각각의 함수 또는 Operator에 대해 Dot을 붙이는 것은 가독성이 떨어지고, 또한 귀찮다. Julia는 식에서 모든 함수와 Operator를 Vectorization해 주는 Macro `@.`를 지원한다.
* **이 Macro `@.`를 Expression 앞에 공백으로 분리하여 붙이면, Julia는 이 Expression에 등장하는 모든 함수와 연산자를 Vectorization하여 실행한다.**

In [87]:
arr1 = [1.0, 2.0, 3.0, 4.0]
arr2 = similar(arr1) # pre-allocate output array
@. arr2 = sin(cos(arr1)) # <=> arr2 .= sin.(cos.(arr1))

4-element Vector{Float64}:
  0.5143952585235492
 -0.4042391538522658
 -0.8360218615377305
 -0.6080830096407656

* 축약 대입 연산자의 경우도 "dot" version의 경우는 대입 연산자까지도 "dot"된 것으로 취급한다.

In [88]:
arr1 = [1.0, 2.0, 3.0, 4.0]
arr2 = [2.0, 3.0, 4.0, 5.0]
arr2 .+= arr1 # <=> arr2 .= arr1 .+ arr2

4-element Vector{Float64}:
 3.0
 5.0
 7.0
 9.0

* Function Chaining도 "dot"화 가능함에 언제나 주의!

In [89]:
1:5 .|> [x -> x^2, inv, x -> 2*x, -, isodd]
# <=>
# [1^2, inv(2), 2*3, -4, isodd(5)]

5-element Vector{Real}:
    1
    0.5
    6
   -4
 true